In [ ]:
import pandas as pd
from pathlib import Path

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing import image

from sklearn.model_selection import train_test_split

from tqdm import tqdm

import os as os
os.getcwd()


In [ ]:
train = pd.read_csv('../input/train.csv')

train.head()


In [ ]:
train_image = []
for i in tqdm(range(train.shape[0])):
    img = image.load_img('../input/train/train/'+ train['id'][i], target_size=(32, 32, 1), 
                         grayscale=False)
    img = image.img_to_array(img)
    img = img/255
    train_image.append(img)
X = np.array(train_image)


In [ ]:
X.shape


In [ ]:
y=train['has_cactus']
y = to_categorical(y)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42,
                                                   test_size=.67)


In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)


In [ ]:
nn1 = Sequential()
nn1.add(Conv2D(32, kernel_size=(3, 3), activation='relu', 
               input_shape=(32,32,3)))
nn1.add(Conv2D(64, (3,3), activation = 'relu'))
nn1.add(MaxPooling2D(pool_size=(2,2)))
nn1.add(Dropout(0.25))
nn1.add(Flatten())
nn1.add(Dense(128, activation='relu'))
nn1.add(Dropout(0.5))
nn1.add(Dense(2, activation='softmax'))


In [ ]:
nn1.compile(loss='categorical_crossentropy',
            optimizer = 'Adam', metrics = ['accuracy'])


In [ ]:
nn1.fit(X_train, y_train, epochs=10, 
        validation_data=(X_test, y_test))


In [ ]:
nn1_score = nn1.evaluate(X_test, y_test, batch_size=128)


In [ ]:
print('Loss ----------------- Accuracy')
print(nn1_score)


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
from keras.utils import np_utils

nn1_pred = nn1.predict(X_test)

nn1_pred_as_class = nn1_pred.argmax(axis=-1)
y_test_as_class = y_test.argmax(axis=-1)

print(classification_report(y_test_as_class, nn1_pred_as_class))


In [ ]:
import glob
from PIL import Image
folder = glob.glob('../input/test/test/*.jpg')
#folder


In [ ]:
Z = np.array([np.array(Image.open(img)) for img in folder])
Z.shape


In [ ]:
sub = nn1.predict_proba(Z)


In [ ]:
sub_df = pd.DataFrame(sub, columns = ['no_cactus','has_cactus'])


In [ ]:
sub_df.head()


In [ ]:
img_names = os.listdir('../input/test/test/')

sub_df['id'] = img_names


In [ ]:
del sub_df['no_cactus']


In [ ]:
sub_df = sub_df[['id', 'has_cactus']]
sub_df.head()


In [ ]:
sub_df.to_csv('sub_1.csv', index=False)
